# Initial Setup and Data Load

In [ ]:
%load_ext autoreload
%autoreload 2
import os

%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from scipy.stats import norm, skew

import math
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

#from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
import category_encoders as ce
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_log_error
from scipy.special import boxcox1p
from scipy.stats import boxcox


import string
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!ls ../input

In [ ]:
PATH = "../input/house-prices-advanced-regression-techniques/"

In [ ]:
df_train=pd.read_csv(f'{PATH}train.csv')#, index_col='Id')
df_test=pd.read_csv(f'{PATH}test.csv')#, index_col='Id')

### Y (target value) to Log, as stated at Kaggle Evaluation page

In [ ]:
# for the purpose of evaluation of current competition we transform target value
df_train.SalePrice = np.log1p(df_train.SalePrice)

In [ ]:
print('Number of Training Examples = {}'.format(df_train.shape[0]))
print('Number of Test Examples = {}\n'.format(df_test.shape[0]))
print('Training X Shape = {}'.format(df_train.shape))
print('Training y Shape = {}\n'.format(df_train['SalePrice'].shape[0]))
print('Test X Shape = {}'.format(df_test.shape))
print('Test y Shape = {}\n'.format(df_test.shape[0]))
#print(df_train.columns)
#print(df_test.columns)

In [ ]:
#print(df_train.info())
#df_train.sample(3)
#print(df_test.info())
#df_test.sample(3)

# Dealing with Outliers

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = df_train['GrLivArea'], y = df_train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show()

In [ ]:
# Deleting outliers
df_train = df_train.drop(df_train[(df_train['GrLivArea']>4000) & (df_train['SalePrice']<300000)].index)

#Check the graphic again
fig, ax = plt.subplots()
ax.scatter(df_train['GrLivArea'], df_train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show()

# DataFrame concatination and Y separation

In [ ]:
def concat_df(train_data, test_data):
    # Returns a concatenated df of training and test set on axis 0
    return pd.concat([train_data, test_data], sort=True).reset_index(drop=True)

df_all = concat_df(df_train, df_test)

df_train.name = 'Training Set'
df_test.name = 'Test Set'
df_all.name = 'All Set' 

dfs = [df_train, df_test]

df_all.shape

In [ ]:
#remember where to divide train and test
ntrain = df_train.shape[0]
ntest = df_test.shape[0]

#Save the 'Id' column
train_ID = df_train['Id']
test_ID = df_test['Id']

In [ ]:
#Dividing Target column (Y)
y_train_full = df_train.SalePrice.values
df_all.drop(['SalePrice'], axis=1, inplace=True)
df_all.drop('Id',axis=1,inplace=True)

# Dealing with Missing Values

### Create columns to mark originally missed values

In [ ]:
def mark_missing (df):
    for col in df.columns:
        if df_all[col].isnull().sum()>0:
            df_all[col+'_missed']=df_all[col].isnull()

In [ ]:
mark_missing(df_all)

In [ ]:
df_all.shape

### Replace Missing

In [ ]:
def display_missing(df):
    for col in df.columns:
        print(col, df[col].isnull().sum())
    print('\n')
    
for df in dfs:
    print(format(df.name))
    display_missing(df)
    
    
    
#Check remaining missing values if any 
def display_only_missing(df):
    all_data_na = (df.isnull().sum() / len(df)) * 100
    all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
    print(missing_data)

In [ ]:
display_only_missing(df_all)

### Replace non-missing but "NA", "None", etc values by Data description

##### Replace NA in Object columns, based on information from description

In [ ]:
# fill NA values (not missed) with None - based on data description -  - for non-Numerical (object) Columns
for col in ('Alley','MasVnrType','BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
            'BsmtFinType2','FireplaceQu','GarageType', 'GarageFinish', 'GarageQual', 
            'GarageCond','PoolQC','Fence','MiscFeature'):
    df_all[col] = df_all[col].fillna('None')

##### Replace NA in Numerical columns, based on information from description

In [ ]:
display_only_missing(df_all)

In [ ]:
#fill NA numerical value with '0' - based on data description of correspondent Object columns - for Numerical Columns
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars','BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath','MasVnrArea'):
    df_all[col] = df_all[col].fillna(0)

### Replacing real missing values

We also have REAL missing values, that we can't just replace based on description that if missed - use 'None' or 0. Hence, we will work here

#### Iteration 1 - replacing by logic and deduction of human

##### Replace NA missing values by most often in column (only for columns with 2 and less NA values, where do not make sense to invest hugely into Analysis)

In [ ]:
display_only_missing(df_all)

In [ ]:
# Fill missing value in corresponding columns with most frequent value in column
for col in ('Utilities','Functional','SaleType','KitchenQual','Exterior2nd','Exterior1st','Electrical'):
    df_all[col].fillna(df_all[col].mode()[0], inplace=True)
    
# Functional : data description says NA means typical
# BTW we just used df_all.Functional.mode() = use most frequent value (as 'Typ' is most frequent value)
#df_all["Functional"] = df_all["Functional"].fillna("Typ")

##### Dealing with missing values left

In [ ]:
display_only_missing(df_all)

##### Dealing with MSZoning

In [ ]:
df_all.MSZoning.isnull().sum()

In MSZoning we have 4 missing values. 
We can replace them either by most common in column, or I have decided just with 'None' object values

In [ ]:
df_all["MSZoning"] = df_all["MSZoning"].fillna("None")

In [ ]:
display_only_missing(df_all)

In [ ]:
#### Iteration 2 - replacing by machine learning

##### Dealing with LotFrontage

In [ ]:
df_all['LotFrontage'].isnull().sum()

In [ ]:
#Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
df_all["LotFrontage"] = df_all.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

In [ ]:
df_all['LotFrontage'].isnull().sum()

In [ ]:
display_only_missing(df_all)

In [ ]:
df_all.info()

##### Seems no missed values
Missing Values = DONE

# Pre-Evaluation - benchmarking before Feature Generation

## Making Training, Validation, Test Dataset

In [ ]:
"""Dividing working DataFrame back to Train and Test"""
# split Validational/Test set from Training set after Categorical Value Engeneering
#X_test=df_all.iloc[ntrain:] # Test set
X_train_full=df_all.iloc[:ntrain] # Train set

In [ ]:
#df_all.shape, y_train_full.shape, X_test.shape, X_train_full.shape

In [ ]:
#X_train, X_valid, y_train, y_valid = train_test_split(pd.get_dummies(X_train_full), y_train_full, random_state=42)

In [ ]:
#X_train.shape, X_valid.shape

## Splitting function (train/valid)

In [ ]:
def quick_get_dumm(df):
    X_train_full=df.iloc[:ntrain] # Full Train set
#    X_test=df_all.iloc[ntrain:] # Test set
    
    # Creating train and validation sets
    X_train, X_valid, y_train, y_valid = train_test_split(pd.get_dummies(X_train_full), y_train_full, random_state=42)
    return X_train, X_valid, y_train, y_valid

In [ ]:
X_train, X_valid, y_train, y_valid = quick_get_dumm(df_all)

In [ ]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape, X_train_full.shape, y_train_full.shape

## Evaluation

In [ ]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m,X_train=X_train, X_valid=X_valid, y_train=y_train, y_valid=y_valid):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

### Experimenting with Random Forest

In [ ]:
m_rf = RandomForestRegressor(n_estimators=160, min_samples_leaf=1, max_features=0.5, n_jobs=-1, oob_score=True, random_state=42)
m_rf.fit(X_train, y_train)
print_score(m_rf)

### Experimenting with Lasso

In [ ]:
def lasso_score(X,y):
    lasso = ElasticNet(random_state=1)
    param = {'l1_ratio' : [0],
             'alpha' : [0.017]}
    lasso = GridSearchCV(lasso, param, cv=5, scoring='neg_mean_squared_error')
    lasso.fit(X,y)
    print('Lasso:', np.sqrt(lasso.best_score_*-1))
    return lasso

In [ ]:
lasso_score(X_train, y_train)

### XGBoost

In [ ]:
m_xgb = XGBRegressor(n_estimators=160, learning_rate=0.05, random_state=42)
# using early_stop to find out where validation scores don't improve
#m_xgb.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_valid, y_valid)], verbose=False)
%time m_xgb.fit(X_train, y_train)
print_score(m_xgb)

# Dealing with missing

We will try ML techniques to predict all real missing values. We'll see how it will improve accuracy

##### Once again dealing with missed LotFrontage feature

In [ ]:
# We created function to return NA values of feature/column back in place, 
# based on _missed column, we created to state what values was missed in original dataset

# returning original NA values back
def return_original_na(df, feature):
    df[feature].loc[df.index[df[feature+'_missed'] == True].tolist()]=np.nan
    return df[feature]

In [ ]:
#Returning original NA values of MSZoning back in place
df_all['LotFrontage']=return_original_na(df_all, 'LotFrontage')

In [ ]:
display_only_missing(df_all)

In [ ]:
def filling_na_with_predictions(df, feature):
    """
    df - DataFrame without target column y. Train+Test DataFrame (df_all)
    feature - feature (column), containing real NA values we will fill

    Assumption:
    All other columns do not have NA values. In case of having we have to impute with some Statistical method (Median, etc)
    We do not do it inside this function
    """

    flag_object=0
    
    if df[feature].isnull().sum()>0:
        ## Store Indexes of rows with NA values (we can just call "_missed" column with True values, to check those indexes as well)
        ## Creating index based on NA values present in column
        na_rows_idxs=df[df[feature].isnull()].index 
            ## Creating index based on NA values being present in original DF column
            #na_rows_idxs=df.index[df[feature+'_missed'] == True].tolist()

        ## For fitting and predictiong - convert DF to dummies DF, ready for ML
        #df=pd.get_dummies(df)
        ## If feature object we cant just dummy all, we shouldn't dummy feature column
        df=pd.concat([ pd.Series(df[feature]), pd.get_dummies(df.drop([feature], axis=1)) ], axis=1)


        ## Splitting DF to Feature_Train_X, Feature_Train_y, Feature_Predict_X:
        ## Feature_Train_X = DF without NA values in "feature_with_NA"column
        ## Feature_Train_y = target values that we have. All values in "feature_with_NA" except NA values
        ## Feature_Predict_X = DF of correcponding to NA values in "feature_with_NA" without target vales (basically because they is equal to NA)
        Feature_Train_X=df.drop(df[df[feature].isnull()].index).drop([feature], axis=1)
        Feature_Train_y=df[feature].drop(df[df[feature].isnull()].index).values
        Feature_Predict_X=df[df[feature].isnull()].drop([feature], axis=1)

        ## If feature is NOT Numerical
        ## Label encoding of y values in case it is not numerical
        if is_string_dtype(df[feature]) or is_categorical_dtype(df[feature]):
            flag_object=1
            from sklearn.preprocessing import LabelEncoder
            le = LabelEncoder()
            le.fit(Feature_Train_y)
            Feature_Train_y=le.transform(Feature_Train_y)
             
        ## Making predictions, what might be in NA fields based on Train DF
        #m_xgb = XGBRegressor(n_estimators=160, learning_rate=0.05)
        #m_xgb.fit(Feature_Train_X, Feature_Train_y)
        lasso = ElasticNet(random_state=1)
        param = {'l1_ratio' : [0],
             'alpha' : [0.017]}
        lasso = GridSearchCV(lasso, param, cv=5, scoring='neg_mean_squared_error')
        lasso.fit(Feature_Train_X,Feature_Train_y)
    
        ## Creating (Predicting) values to impute NA
        #fillna_values=m_xgb.predict(Feature_Predict_X)
        fillna_values=lasso.predict(Feature_Predict_X)

        ## If feature is NOT Numerical
        ## Return Encoded values back to Object/Category if feature NOT numerical
        if flag_object==1:
            fillna_values=le.inverse_transform(np.around(fillna_values).astype(int))
        
        ## Replacing NA values with predicted Series of values
        df[feature]=df[feature].fillna(pd.Series(index=na_rows_idxs,data=fillna_values))

        ## Returning feature column without NA values    
        return df[feature]
    else:
        print ('There were no NA values')

In [ ]:
df_all['LotFrontage']=filling_na_with_predictions(df_all, "LotFrontage")

In [ ]:
def evaluate(df):
    # Split dataset for train-validation
    X_train, X_valid, y_train, y_valid = quick_get_dumm(df)
    
    #Lasso
    lasso_score(X_train, y_train)

    #XGBoost
    m_xgb.fit(X_train, y_train)
    print('XGBoost')
    print_score(m_xgb,X_train, X_valid, y_train, y_valid)

    # Random Forest
    m_rf.fit(X_train, y_train)
    print('Random Forest')
    print_score(m_rf,X_train, X_valid, y_train, y_valid)

In [ ]:
evaluate(df_all)

Great! As we can see in all 3 models scores improved using ML algorithms to replace missing values

##### Once again dealing with missed MSZoning feature

In [ ]:
#Returning original NA values of MSZoning back in place
df_all['MSZoning']=return_original_na(df_all, 'MSZoning')

In [ ]:
display_only_missing(df_all)

In [ ]:
df_all[df_all['MSZoning'].isnull()].index

In [ ]:
df_all['MSZoning']=filling_na_with_predictions(df_all, 'MSZoning')

In [ ]:
df_all['MSZoning'].loc[df_all.index[df_all['MSZoning'+'_missed'] == True].tolist()]

As we can see we had all 'RL' values for MSZoning column, but ML algorithms proposed to change it a little bit. Let's check score

In [ ]:
evaluate(df_all)

In [ ]:
##### Dealing with Missing values we replaced with most common - now replacing them with ML predictions

In [ ]:
for col in ('Utilities','Functional','SaleType','KitchenQual','Exterior2nd','Exterior1st','Electrical'):
    print ('Filling with most common:\n',df_all[col].loc[df_all.index[df_all[col+'_missed'] == True].tolist()])
    df_all[col]=return_original_na(df_all, col)
    df_all[col]=filling_na_with_predictions(df_all, col)
    print ('Filling with predictions:\n',df_all[col].loc[df_all.index[df_all[col+'_missed'] == True].tolist()])

In [ ]:
evaluate(df_all)

As we can see - nothing in scores changed, so it was unnecessary step, possibly because these last features weren't important for models

# Feature Importance


In [ ]:
fi = pd.DataFrame({'feature': list(X_train.columns), 'importance':m_rf.feature_importances_}).sort_values('importance',ascending=False)

In [ ]:
fi[:50]

# Label Encoding

In [ ]:
# Deprecated, unnessesary
def select_encoding (df_all,encoding='onehot'):
    if encoding=='label':
        # Label Encoding
        cols=[]
        cols.extend(ordinal_features)
        cols.extend(categorical_features)
        cols.extend(df_all.select_dtypes(object).columns)
        # process columns, apply LabelEncoder to categorical features
        for c in cols:
            if c in df_all.columns:
                lbl = LabelEncoder() 
                lbl.fit(list(df_all[c].values)) 
                df_all[c] = lbl.transform(list(df_all[c].values))
    if encoding=='binary':
        # Binary Encoding
        cols=[]
        #cols.extend(ordinal_features)
        cols.extend(categorical_features)
        cols.extend(df_all.select_dtypes(object).columns)
        # process columns, apply BinaryEncoder to categorical features
        for c in cols:
            if c in df_all.columns:
                bnr = ce.binary.BinaryEncoder() 
                bnr.fit(list(df_all[c].values)) 
                df_all[c] = bnr.transform(list(df_all[c].values))
    if encoding=='onehot':
        df_all=pd.get_dummies(df_all)
    return df_all

In [ ]:
def encoding_check_score(df):
    X_train, X_valid, y_train, y_valid=quick_get_dumm(df)
    # Lasso
    print ('Lasso Score: ')
    lasso_score(X_train, y_train)

"""    # Random Forest
    m_rf.fit(X_train, y_train)
    print ('Random Forest Score: ')#; print_score(m_rf)
    res = [rmse(m_rf.predict(X_train), y_train), rmse(m_rf.predict(X_valid), y_valid),
                m_rf.score(X_train, y_train), m_rf.score(X_valid, y_valid)]
    if hasattr(m_rf, 'oob_score_'): res.append(m_rf.oob_score_)
    print(res)
    
    # XGBoost
    m_xgb.fit(X_train, y_train)
    print ('XGBoost Score: ')#; print_score(m_xgb)
    res = [rmse(m_xgb.predict(X_train), y_train), rmse(m_xgb.predict(X_valid), y_valid),
                m_xgb.score(X_train, y_train), m_xgb.score(X_valid, y_valid)]
    #if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    
    print(res)
"""

In [ ]:
def encoding_measure (df, feature):
    enc=['ordinal','onehot','label','binary']#, 'BackwardDifferenceEncoder','HashingEncoder','HelmertEncoder','PolynomialEncoder']#'LeaveOneOutEncoder','TargetEncoder','WOEEncoder',
    for encoding in enc:
        if encoding=='ordinal':
        # As Is encoding
            df_ordinal=df.copy()
            print (feature, 'Ordinal Encoding')
            encoding_check_score(df_ordinal)
        if encoding=='onehot':
        # OneHot encoding
            df_onehot=df.copy()
            df_onehot[feature]=df_onehot[feature].astype(str)
            df_onehot=pd.get_dummies(df_onehot)
            print (feature, 'OneHot Encoding')
            encoding_check_score(df_onehot)
        if encoding=='label':
        # Label Encoding
            df_le=df.copy()
            df_le[feature]=df_le[feature].astype(str)
            lbl = LabelEncoder() 
            lbl.fit(list(df_le[feature].values)) 
            df_le[feature] = lbl.transform(list(df_le[feature].values))
            print (feature, 'Label Encoding')
            encoding_check_score(df_le)
        if encoding=='binary':
        # Binary Encoding
            df_be=df.copy()
            df_be[feature]=df_be[feature].astype(str)
            bnr = ce.binary.BinaryEncoder() 
            bnr.fit(list(df_be[feature].values)) 
            df_be[feature] = bnr.transform(list(df_be[feature].values))
            print (feature, 'Binary Encoding')
            encoding_check_score(df_be)
        if encoding=='LeaveOneOutEncoder':
        # LeaveOneOutEncoder
        #category_encoders.leave_one_out.LeaveOneOutEncoder
            df_loo=df.copy()
            df_loo[feature]=df_loo[feature].astype(str)
            loo = ce.leave_one_out.LeaveOneOutEncoder() 
#            X_train_full=df_loo.iloc[:ntrain] # Train set
            loo.fit(list(df_loo[feature].values),y_train_full) 
            df_loo[feature] = loo.transform(list(df_loo[feature].values))
            print (feature, 'LeaveOneOutEncoder')
            encoding_check_score(df_loo)
        if encoding=='BackwardDifferenceEncoder':
        # Backward Difference Coding
        #category_encoders.backward_difference.BackwardDifferenceEncoder
            df=df.copy()
            df[feature]=df[feature].astype(str)
            enc = ce.hashing.HashingEncoder() 
            enc.fit(list(df[feature].values)) 
            df[feature] = enc.transform(list(df[feature].values))
            print (feature, encoding)
            encoding_check_score(df)
        if encoding=='HashingEncoder':
        # Hashing
        #category_encoders.hashing.HashingEncoder
            df=df.copy()
            df[feature]=df[feature].astype(str)
            enc = ce.hashing.HashingEncoder() 
            enc.fit(list(df[feature].values)) 
            df[feature] = enc.transform(list(df[feature].values))
            print (feature, encoding)
            encoding_check_score(df)
        if encoding=='HelmertEncoder':
        # Helmert
        #category_encoders.helmert.HelmertEncoder
            df=df.copy()
            df[feature]=df[feature].astype(str)
            enc = ce.helmert.HelmertEncoder() 
            enc.fit(list(df[feature].values)) 
            df[feature] = enc.transform(list(df[feature].values))
            print (feature, encoding)
            encoding_check_score(df)        
        if encoding=='PolynomialEncoder':
        # Polinomial Coding
        #category_encoders.polynomial.PolynomialEncoder
            df=df.copy()
            df[feature]=df[feature].astype(str)
            enc = ce.polynomial.PolynomialEncoder() 
            enc.fit(list(df[feature].values)) 
            df[feature] = enc.transform(list(df[feature].values))
            print (feature, encoding)
            encoding_check_score(df)        
        if encoding=='TargetEncoder':
        # Target
        #category_encoders.target_encoder.TargetEncoder
            df=df.copy()
            df[feature]=df[feature].astype(str)
            enc = ce.target_encoder.TargetEncoder() 
#            X_train_full=df.iloc[:ntrain] # Train set
            enc.fit(list(df[feature].values),y_train_full) 
            df[feature] = enc.transform(list(df[feature].values))
            print (feature, encoding)
            encoding_check_score(df)        
        if encoding=='WOEEncoder':
        #Weight of Evidence
        #category_encoders.woe.WOEEncoder
            df=df.copy()
            df[feature]=df[feature].astype(str)
            enc = ce.woe.WOEEncoder() 
            X_train_full=df.iloc[:ntrain] # Train set
            enc.fit(list(X_train_full[feature].values),y_train_full) 
            df[feature] = enc.transform(list(df[feature].values))
            print (feature, encoding, df.feature)
#            encoding_check_score(df)
        
#        print ('\n\n')
        #return df

# Dealing with Ordinal values

## Ordinal Data Encoding

### Encoding quality columns with dictionary

In [ ]:
""""
Encode Quality columns with:
Ex	Excellent
Gd	Good
TA	Average/Typical
Fa	Fair
Po	Poor
NA	No "Garage/Basement/Fireplace/..."

To decode we use same Disctionary as used in other dataset columns:
OverallCond: Rates the overall condition of the house
       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average	
       5	Average
       4	Below Average	
       3	Fair
       2	Poor
       1	Very Poor
"""

qual_cleanup = {"Ex": 9, "Gd": 7, "TA": 5, "Fa": 3,"Po": 2, "None": 0}

# Checking/Evaluation effectiveness (error) of different encoding approaches (AsIs, OneHot, Label, Binary)
for col in ('ExterQual','ExterCond','BsmtQual','BsmtCond','HeatingQC','KitchenQual',
            'FireplaceQu','GarageQual','GarageCond','PoolQC'):
    df_all_tmp=df_all.copy()
    df_all_tmp[col].replace(qual_cleanup, inplace=True)
    df_all_tmp[col]=df_all_tmp[col].astype(float)
    encoding_measure (df_all_tmp, feature=col)
    print ('---------------------------')

In [ ]:
onehot_features=[]
onehot_features.append('KitchenQual')
onehot_features.append('PoolQC')
label_features=[]
label_features.append('GarageQual')

In [ ]:
""""
BsmtFinType1: Rating of basement finished area
BsmtFinType2: Rating of basement finished area (if multiple types)
       GLQ	Good Living Quarters
       ALQ	Average Living Quarters
       BLQ	Below Average Living Quarters	
       Rec	Average Rec Room
       LwQ	Low Quality
       Unf	Unfinshed
       NA	No Basement
"""

qual_cleanup = {"GLQ": 10, "ALQ": 8, "BLQ": 6, "Rec": 4, "LwQ": 3,"Unf": 2, "None": 0}

# Checking/Evaluation effectiveness (error) of different encoding approaches (AsIs, OneHot, Label, Binary)
for col in ('BsmtFinType1','BsmtFinType2'):
    df_all_tmp=df_all.copy()
    df_all_tmp[col].replace(qual_cleanup, inplace=True)
    df_all_tmp[col]=df_all_tmp[col].astype(float)    
    encoding_measure (df_all_tmp, feature=col)
    print ('---------------------------')

In [ ]:
label_features.append('BsmtFinType2')

In [ ]:
"""
BsmtExposure: Refers to walkout or garden level walls
       Gd	Good Exposure
       Av	Average Exposure (split levels or foyers typically score average or above)	
       Mn	Mimimum Exposure
       No	No Exposure
       NA	No Basement
"""
qual_cleanup = {"Gd": 10, "Av": 7, "Mn": 4, "No": 2, "None": 0}

# Checking/Evaluation effectiveness (error) of different encoding approaches (AsIs, OneHot, Label, Binary)
df_all_tmp=df_all.copy()
df_all_tmp['BsmtExposure'].replace(qual_cleanup, inplace=True)
df_all_tmp['BsmtExposure']=df_all_tmp['BsmtExposure'].astype(float)
encoding_measure (df_all_tmp, feature='BsmtExposure')
print ('---------------------------')

#### Working on Functional (seems decrease score, not used now)

In [ ]:
""""
Functional: Home functionality (Assume typical unless deductions are warranted)
       Typ	Typical Functionality
       Min1	Minor Deductions 1
       Min2	Minor Deductions 2
       Mod	Moderate Deductions
       Maj1	Major Deductions 1
       Maj2	Major Deductions 2
       Sev	Severely Damaged
       Sal	Salvage only

"""

qual_cleanup = {"Typ": 10, "Min1": 9, "Min2": 8, "Mod": 6, "Maj1": 4,"Maj2": 3, "Sev": 1, "Sal": 0}

# Checking/Evaluation effectiveness (error) of different encoding approaches (AsIs, OneHot, Label, Binary)
df_all_tmp=df_all.copy()
df_all_tmp['Functional'].replace(qual_cleanup, inplace=True)
df_all_tmp['Functional']=df_all_tmp['Functional'].astype(float)
encoding_measure (df_all_tmp, feature='Functional')
print ('---------------------------')

In [ ]:
"""
GarageFinish: Interior finish of the garage

       Fin	Finished
       RFn	Rough Finished	
       Unf	Unfinished
       NA	No Garage
"""

qual_cleanup = {"Fin": 10, "RFn": 7, "Unf": 4, "None": 0}

# Checking/Evaluation effectiveness (error) of different encoding approaches (AsIs, OneHot, Label, Binary)
df_all_tmp=df_all.copy()
df_all_tmp['GarageFinish'].replace(qual_cleanup, inplace=True)
df_all_tmp['GarageFinish']=df_all_tmp['GarageFinish'].astype(float)
encoding_measure (df_all_tmp, feature='GarageFinish')
print ('---------------------------')

In [ ]:
label_features.append('GarageFinish')

In [ ]:
"""
GarageType: Garage location
		
       2Types	More than one type of garage
       Attchd	Attached to home
       Basment	Basement Garage
       BuiltIn	Built-In (Garage part of house - typically has room above garage)
       CarPort	Car Port
       Detchd	Detached from home
       NA	No Garage
"""


qual_cleanup = {"2Types": 10, "Attchd": 8, "Basment": 6, "BuiltIn": 4, "CarPort": 3,"Detchd": 2, "None": 0}

# Checking/Evaluation effectiveness (error) of different encoding approaches (AsIs, OneHot, Label, Binary)
df_all_tmp=df_all.copy()
df_all_tmp['GarageType'].replace(qual_cleanup, inplace=True)
df_all_tmp['GarageType']=df_all_tmp['GarageType'].astype(float)
encoding_measure (df_all_tmp, feature='GarageType')
print ('---------------------------')

In [ ]:
label_features.append('GarageType')

##### Dealing with BldgType

In [ ]:
df_all['BldgType'].value_counts()

In [ ]:
"""
BldgType: Type of dwelling
       1Fam	Single-family Detached	
       2FmCon	Two-family Conversion; originally built as one-family dwelling
       Duplx	Duplex
       TwnhsE	Townhouse End Unit
       TwnhsI	Townhouse Inside Unit
"""

qual_cleanup = {"Twnhs": 5, "TwnhsE": 4, "Duplex": 3, "2fmCon": 2, "1Fam": 1}

# Checking/Evaluation effectiveness (error) of different encoding approaches (AsIs, OneHot, Label, Binary)
df_all_tmp=df_all.copy()
df_all_tmp['BldgType'].replace(qual_cleanup, inplace=True)
df_all_tmp['BldgType']=df_all_tmp['BldgType'].astype(float)
encoding_measure (df_all_tmp, feature='BldgType')
print ('---------------------------')

##### Dealing with HouseStyle

In [ ]:
df_all['HouseStyle'].value_counts()

In [ ]:
""""
HouseStyle: Style of dwelling
       1Story	One story
       1.5Fin	One and one-half story: 2nd level finished
       1.5Unf	One and one-half story: 2nd level unfinished
       2Story	Two story
       2.5Fin	Two and one-half story: 2nd level finished
       2.5Unf	Two and one-half story: 2nd level unfinished
       SFoyer	Split Foyer
       SLvl	Split Level
"""
qual_cleanup = {"SFoyer":8,"SLvl":7,"2.5Fin":6,"2.5Unf": 5, "2Story": 4, "1.5Fin": 3, "1.5Unf": 2, "1Story": 1}

# Checking/Evaluation effectiveness (error) of different encoding approaches (AsIs, OneHot, Label, Binary)
df_all_tmp=df_all.copy()
df_all_tmp['HouseStyle'].replace(qual_cleanup, inplace=True)
df_all_tmp['HouseStyle']=df_all_tmp['HouseStyle'].astype(float)
encoding_measure (df_all_tmp, feature='HouseStyle')
print ('---------------------------')

##### Dealing with Electrical

In [ ]:
df_all['Electrical'].value_counts()

In [ ]:
""""
Electrical: Electrical system
       SBrkr	Standard Circuit Breakers & Romex
       FuseA	Fuse Box over 60 AMP and all Romex wiring (Average)	
       FuseF	60 AMP Fuse Box and mostly Romex wiring (Fair)
       FuseP	60 AMP Fuse Box and mostly knob & tube wiring (poor)
       Mix	Mixed
"""
qual_cleanup = {"SBrkr": 5, "FuseA": 4, "FuseF": 3, "FuseP": 2, "Mix": 1}

# Checking/Evaluation effectiveness (error) of different encoding approaches (AsIs, OneHot, Label, Binary)
df_all_tmp=df_all.copy()
df_all_tmp['Electrical'].replace(qual_cleanup, inplace=True)
df_all_tmp['Electrical']=df_all_tmp['Electrical'].astype(float)
encoding_measure (df_all_tmp, feature='Electrical')
print ('---------------------------')

#### Chosing encoding for ordinal data

In [ ]:
ordinal_features=[]
categorical_features=[]

In [ ]:
# Chosing Ordinal Encoding for Ordinal Data as most effective
qual_cleanup = {"Ex": 9, "Gd": 7, "TA": 5, "Fa": 3,"Po": 2, "None": 0}

#Ordinal encoding
for col in ('ExterQual','ExterCond','BsmtQual','BsmtCond','KitchenQual',
            'HeatingQC','FireplaceQu','GarageQual','GarageCond','PoolQC'):
    ordinal_features.append(col)
    df_all[col].replace(qual_cleanup, inplace=True)
    df_all[col]=df_all[col].astype(float)


In [ ]:
# Chosing Ordinal Encoding for Ordinal Data as most effective
qual_cleanup = {"GLQ": 10, "ALQ": 8, "BLQ": 6, "Rec": 4, "LwQ": 3,"Unf": 2, "None": 0}

for col in ('BsmtFinType1','BsmtFinType2'):
    ordinal_features.append(col)
    df_all[col].replace(qual_cleanup, inplace=True)
    df_all[col]=df_all[col].astype(float)    

In [ ]:
# Chosing Ordinal Encoding for Ordinal Data as most effective
qual_cleanup = {"Gd": 10, "Av": 7, "Mn": 4, "No": 2, "None": 0}

ordinal_features.append('BsmtExposure')
df_all['BsmtExposure'].replace(qual_cleanup, inplace=True)
df_all['BsmtExposure']=df_all['BsmtExposure'].astype(float)

In [ ]:
# Chosing Ordinal Encoding for Ordinal Data as most effective
qual_cleanup = {"Typ": 10, "Min1": 9, "Min2": 8, "Mod": 6, "Maj1": 4,"Maj2": 3, "Sev": 1, "Sal": 0}

ordinal_features.append('Functional')
df_all['Functional'].replace(qual_cleanup, inplace=True)
df_all['Functional']=df_all['Functional'].astype(float)

In [ ]:
# Chosing Ordinal Encoding for Ordinal Data as most effective
qual_cleanup = {"Fin": 10, "RFn": 7, "Unf": 4, "None": 0}

ordinal_features.append('GarageFinish')
df_all['GarageFinish'].replace(qual_cleanup, inplace=True)
df_all['GarageFinish']=df_all['GarageFinish'].astype(float)

In [ ]:
# Chosing Ordinal Encoding for Ordinal Data as most effective
qual_cleanup = {"2Types": 10, "Attchd": 8, "Basment": 6, "BuiltIn": 4, "CarPort": 3,"Detchd": 2, "None": 0}

ordinal_features.append('GarageType')
df_all['GarageType'].replace(qual_cleanup, inplace=True)
df_all['GarageType']=df_all['GarageFinish'].astype(float)

In [ ]:
# Chosing Ordinal Encoding for Ordinal Data as most effective
qual_cleanup = {"SFoyer":8,"SLvl":7,"2.5Fin":6,"2.5Unf": 5, "2Story": 4, "1.5Fin": 3, "1.5Unf": 2, "1Story": 1}

ordinal_features.append('HouseStyle')
df_all['HouseStyle'].replace(qual_cleanup, inplace=True)
df_all['HouseStyle']=df_all['HouseStyle'].astype(float)

In [ ]:
# Chosing Ordinal Encoding for Ordinal Data as most effective
qual_cleanup = {"SBrkr": 5, "FuseA": 4, "FuseF": 3, "FuseP": 2, "Mix": 1}

ordinal_features.append('Electrical')
df_all['Electrical'].replace(qual_cleanup, inplace=True)
df_all['Electrical']=df_all['Electrical'].astype(float)

In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

In [ ]:
ordinal_features

In [ ]:
categorical_features

In [ ]:
label_features

In [ ]:
onehot_features

In [ ]:
lbl = LabelEncoder() 
for feature in label_features:
    # Label Encoding
#    df_all[feature]=df_all[feature].astype(str)  
#    lbl.fit(list(df_all[feature].values)) 
#    df_all[feature] = lbl.fit_transform(list(df_all[feature].values))

In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

In [ ]:
df_all.GarageQual

In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

# Dealing with Categorical values

In [ ]:
def show_object_columns(df):
    for col in df:
        if is_string_dtype(df[col]):
            print(col)

In [ ]:
show_object_columns(df_all)

In [ ]:
categorical_features=[]

In [ ]:
df_all.MSSubClass.head()

In [ ]:
# Transforming some numerical variables that are really categorical

# MSSubClass=The building class
"""
MSSubClass: Identifies the type of dwelling involved in the sale.	
        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES
"""
encoding_measure (df_all, feature='MSSubClass')

In [ ]:
df_all['MSSubClass'] = df_all['MSSubClass'].astype(str)
categorical_features.append('MSSubClass')

In [ ]:
df_all.MSSubClass.head()

In [ ]:
# Changing OverallCond into a categorical variable
"""
OverallCond: Rates the overall condition of the house
       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average	
       5	Average
       4	Below Average	
       3	Fair
       2	Poor
       1	Very Poor
"""
encoding_measure (df_all, feature='OverallCond')

In [ ]:
# Changing OverallQual into a categorical variable
"""
OverallQual: Rates the overall material and finish of the house
       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average
       5	Average
       4	Below Average
       3	Fair
       2	Poor
       1	Very Poor
"""
encoding_measure (df_all, feature='OverallQual')

In [ ]:
## still under question how to encode MoSold
# Year and month sold are transformed into categorical features.
#df_all['YrSold'] = df_all['YrSold'].astype(str)
#df_all['MoSold'] = df_all['MoSold'].astype(str)
#categorical_features.append('YrSold')
encoding_measure (df_all, feature='YrSold')
#categorical_features.append('MoSold')
encoding_measure (df_all, feature='MoSold')

In [ ]:
#df_all['YearBuilt']=df_all['YearBuilt'].astype(str)
#categorical_features.append('YearBuilt')
encoding_measure (df_all, feature='YearBuilt')

#df_all['YearRemodAdd']=df_all['YearRemodAdd'].astype(str)
#categorical_features.append('YearRemodAdd')
encoding_measure (df_all, feature='YearRemodAdd')

#df_all['GarageYrBlt']=df_all['GarageYrBlt'].astype(str)
#categorical_features.append('GarageYrBlt')
encoding_measure (df_all, feature='GarageYrBlt')

In [ ]:
df_all.info(all)

# Scewed data

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
skewness = df_all.select_dtypes(include=numerics).apply(lambda x: skew(x))
skew_index = skewness[abs(skewness) >= 0.75].index
skewness[skew_index].sort_values(ascending=False)

In [ ]:
'''BoxCox Transform'''
lam = 0.15
for column in skew_index:
    df_all[column] = boxcox1p(df_all[column], lam)
    #continue

In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

## Check numeric columns (if they are actually Categorical, like Year)

### Experimenting - heavily convert NUMERICAL to CATEGORICAL

# using list of quntative and qualitative

In [ ]:
df_all.select_dtypes(object).columns

In [ ]:
ordinal_features

In [ ]:
categorical_features

# Feature Importance Dropping

### Finding which Features to Drop by function and visualisation

# Features generation

In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

In [ ]:
df_all['Age_Build']=df_all['YrSold'].astype(int)-df_all['YearBuilt'].astype(int)

In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

In [ ]:
#df_all.drop(['Age_Build'],axis=1,inplace=True)

In [ ]:
df_all['Age_Remod']=df_all['YrSold'].astype(int)-df_all['YearRemodAdd'].astype(int)

In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

In [ ]:
#df_all.drop(['Age_Remod'],axis=1,inplace=True)

In [ ]:
df_all['Sizes_Total']=df_all['GrLivArea']+df_all['GarageCars']+df_all['GarageArea']+df_all['TotalBsmtSF']+df_all['1stFlrSF']+df_all['2ndFlrSF']+df_all['OpenPorchSF']+df_all['MasVnrArea']


In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

In [ ]:
#df_all.drop(['Quantity_Total'],axis=1,inplace=True)

In [ ]:
df_all['Garage_Age_Build']=df_all['YrSold'].astype(float)-df_all['GarageYrBlt'].astype(float)

In [ ]:
df_all['YrBltAndRemod']=df_all['YearBuilt']+df_all['YearRemodAdd']

In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

In [ ]:
#df_all.drop(['YrBltAndRemod'],axis=1,inplace=True)

In [ ]:
df_all['TotalSF']=df_all['TotalBsmtSF'] + df_all['1stFlrSF'] + df_all['2ndFlrSF']

In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

In [ ]:
df_all['Total_sqr_footage'] = (df_all['BsmtFinSF1'] + df_all['BsmtFinSF2'] +
                                 df_all['1stFlrSF'] + df_all['2ndFlrSF'])

In [ ]:
df_all['Total_Bathrooms'] = (df_all['FullBath'] + (0.5 * df_all['HalfBath']) +
                               df_all['BsmtFullBath'] + (0.5 * df_all['BsmtHalfBath']))

In [ ]:
df_all['Total_porch_sf'] = (df_all['OpenPorchSF'] + df_all['3SsnPorch'] +
                              df_all['EnclosedPorch'] + df_all['ScreenPorch'] +
                              df_all['WoodDeckSF'])

In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

In [ ]:
#df_all.drop(['haspool'],axis=1,inplace=True)

In [ ]:
df_all['has2ndfloor'] = df_all['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

In [ ]:
df_all['hasgarage'] = df_all['GarageArea'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
df_all['hasbsmt'] = df_all['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
df_all['hasfireplace'] = df_all['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

In [ ]:
#df_all.drop(['hasfireplace'],axis=1,inplace=True)

In [ ]:
df_all['Basement']=(df_all['TotalBsmtSF']+df_all['BsmtFinSF1']+df_all['BsmtFinSF2']-df_all['BsmtUnfSF'])*(df_all['BsmtQual']+df_all['BsmtCond'].astype(int)+df_all['BsmtFinType1']+df_all['BsmtExposure'].astype(int)+df_all['BsmtFinType2'])


In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

In [ ]:
#House=

In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

# Data examining

In [ ]:
df_all.info(all)

In [ ]:
df_all.select_dtypes(object).columns

## Housing Crisis Data 2008-2009

#### Shiller index Monthly

# Label Encoding

# Dummies

In [ ]:
df_all=pd.get_dummies(df_all)

In [ ]:
df_all.shape

In [ ]:
df_all.columns

In [ ]:
"""Dividing working DataFrame back to Train and Test"""
# split Validational/Test set from Training set after Categorical Value Engeneering
#def original_train_test(df_all):
X_test=df_all.iloc[ntrain:] # Test set
X_train_full=df_all.iloc[:ntrain] # Train set

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(pd.get_dummies(X_train_full), y_train_full)

In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

# Dropping low variance features

In [ ]:
# Saving all features for future comparison.
all_features = df_all.keys()
# Removing features.
df_all = df_all.drop(df_all.loc[:,(df_all==0).sum()>=(df_all.shape[0]*0.992)],axis=1)
df_all = df_all.drop(df_all.loc[:,(df_all==1).sum()>=(df_all.shape[0]*0.992)],axis=1) 
# Getting and printing the remaining features.
remain_features = df_all.keys()
remov_features = [st for st in all_features if st not in remain_features]
print(len(remov_features), 'features were removed:', remov_features)

In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

# Normalization

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.RobustScaler()
df_all = pd.DataFrame(scaler.fit_transform(df_all))

In [ ]:
X_train, X_valid, y_train, y_valid=quick_get_dumm(df_all)
lasso_score(X_train, y_train)

# Machine Learning

In [ ]:
#df_all = df_all.reset_index()

In [ ]:
# Remove all ,[] symbols from dataframe columns and values
#df_all.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in df_all.columns.values]

In [ ]:
"""Dividing working DataFrame back to Train and Test"""
# split Validational/Test set from Training set after Categorical Value Engeneering
#def original_train_test(df_all):
X_test=df_all.iloc[ntrain:] # Test set
X_train_full=df_all.iloc[:ntrain] # Train set

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(pd.get_dummies(X_train_full), y_train_full)

In [ ]:
m_xgb.fit(X_train, y_train)
print_score(m_xgb)

In [ ]:
def cv_train():
    lasso = ElasticNet(random_state=1)
    param = {'l1_ratio' : [0],
             'alpha' : [0.017]}
    lasso = GridSearchCV(lasso, param, cv=5, scoring='neg_mean_squared_error')
    lasso.fit(X_train_full, y_train_full)
    print('Lasso:', np.sqrt(lasso.best_score_*-1))
    return lasso
lasso = cv_train()

# Predictions for submission

In [ ]:
y_pred=np.expm1(lasso.predict(X_test))

In [ ]:
y_pred

# Submission

In [ ]:
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['SalePrice'] = y_pred
sub.to_csv('submittions/submission_31Aug19.csv',index=False)

In [ ]:
sub.head()

# Stacking